In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals
import tensorflow as tf
import numpy as np
import copy
import tqdm
import IProgress
from hfunc import models
from hfunc import metrics

In [2]:
mnist = tf.keras.datasets.mnist
class_accuracy = metrics.ClassAccuracy()

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [ ]:
model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test))

In [ ]:
old = model.get_weights()

In [ ]:
n = 2
best_weights, _, nodes_removed = remove_nodes(acc, loss, old, n, 50)

new_weights = [np.zeros((best_weights[0].shape[0], best_weights[0].shape[1] - n)), np.zeros((best_weights[1].shape[0] - n)), np.zeros((best_weights[2].shape[0] - n, best_weights[2].shape[1])), best_weights[3]]

j = 0
for i in range(len(best_weights[1])):
    if i not in nodes_removed:
        new_weights[0][:, j] = best_weights[0][:, i]
        new_weights[1][j] = best_weights[1][i]
        new_weights[2][j, :] = best_weights[2][i, :]
        j = j + 1
    
new_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128 - n, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
new_model.set_weights(new_weights)

In [ ]:
def shrink_model(model, shrinkage_factor, x_train, y_train, size, to_test, v=0):
    
    n = shrinkage_factor
    loss, acc = model.evaluate(x_train, y_train, verbose=2)
    old = model.get_weights()
    best_weights, _, nodes_removed = remove_nodes(acc, loss, old, n, to_test, x_train, y_train, v)

    new_weights = [np.zeros((best_weights[0].shape[0], best_weights[0].shape[1] - n)), np.zeros((best_weights[1].shape[0] - n)), np.zeros((best_weights[2].shape[0] - n, best_weights[2].shape[1])), best_weights[3]]

    j = 0
    for i in range(len(best_weights[1])):
        if i not in nodes_removed:
            new_weights[0][:, j] = best_weights[0][:, i]
            new_weights[1][j] = best_weights[1][i]
            new_weights[2][j, :] = best_weights[2][i, :]
            j = j + 1

    new_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(size - n, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    new_model.set_weights(new_weights)
    return new_model, size-n

In [ ]:
new_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
new_model.fit(x_train, y_train, epochs=3)

In [ ]:
new_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
size = 128
to_test = 25
for _ in range(4):
    model.fit(x_train, y_train, epochs=1)
    model, size = shrink_model(model, 8, x_train, y_train, size, to_test)
    print(len(model.get_weights()[1]))
model.fit(x_train, y_train, epochs=1)
model.evaluate(x_test, y_test, verbose=2)
